# CONNECT THE BOARD

In [34]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [ ]:
f = open("main.py", "r")
print(f.read())

In [ ]:
%sendtofile lib/collector.py --source lib/collector.py
%sendtofile lib/peripherals.py --source lib/peripherals.py
%sendtofile lib/signal.py --source lib/signal.py

In [36]:
%sendtofile main.py --source main.py

Sent 69 lines (1617 bytes) to main.py.


# COLLECTOR METHOD

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

from lib.requests import MicroWebCli as requests
import ujson as json

from lib.utils import connect_wifi, load_env_vars

from ulab import numpy as np
import utime as time
from lib.collector import Collector 
from lib.runner import Runner 

In [ ]:
collector = Collector(buffer_size=256, preprocess=True, ds=True)

SPI_params= {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,
}

collector.setup(spi_params=SPI_params)

# ssid = 'TP-Link_AP_4C04'
# password = '63525465'
# connect_wifi(ssid, password)
time.sleep(4)

collector.run()

time.sleep(10)

while True:
    time.sleep(1)
    bt = time.time()
    data = collector.read_output_buffer()
    et = time.time()
    print(et-bt,data)
#     toSend = {"raw_data":data}
#     print(toSend)
#     requests.JSONRequest("http://192.168.0.13:5001/collect", toSend)
#     # requests.JSONRequest("http://172.20.10.2:5001/collect", toSend)
#     del data
#     del toSend
#     # requests.GETRequest("http://172.20.10.2:5001/save")
#     requests.GETRequest("http://192.168.0.13:5001/save")

In [ ]:
collector.stop()

In [ ]:
runner = Runner('CCA', buffer_size=256)
runner.setup()

# ssid = 'VM0857951'
# password = 'cchk7mrYgrzq'
# connect_wifi(ssid, password)

time.sleep(4)
runner.run()
while True:
    time.sleep(1)
    data = runner.read_output_buffer()
    print(data)

In [ ]:
runner.stop()

# DIRECTLY READ ADC, SKIP EVERYTHING ELSE

In [30]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [31]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 100,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    **temp_spi_params
)

data = bytearray([17, 200])
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

# print(adc_sample)
# print("20000 ADC readings done after %u us." %(t2-t1))
# print("Mean time for each ADC reading = %15.13f us" % ((t2-t1)/20000.0))
# print("ADC reading = %15.13f ksamples/s" % (1000/((t2-t1)/20000.0)))

connecting to network...
network config: ('192.168.0.84', '255.255.255.0', '192.168.0.1', '194.168.4.100')


In [32]:
while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

70896
{'raw_data': [1710, 1703, 1717, 1744, 1755, 1737, 1702, 1712, 1742, 1763, 1744, 1707, 1707, 1740, 1765, 1750, 1716, 1711, 1733, 1758, 1751, 1715, 1706, 1729, 1759, 1757, 1718, 1705, 1726, 1751, 1755, 1723, 1705, 1723, 1753, 1761, 1730, 1707, 1722, 1749, 1765, 1735, 1706, 1718, 1744, 1764, 1741, 1708, 1713, 1741, 1767, 1745, 1714, 1713, 1740, 1757, 1744, 1712, 1711, 1733, 1762, 1751, 1718, 1708, 1728, 1760, 1755, 1719, 1702, 1724, 1757, 1758, 1727, 1722, 1724, 1744, 1750, 1724, 1699, 1715, 1745, 1761, 1733, 1702, 1715, 1744, 1761, 1719, 1706, 1715, 1744, 1767, 1744, 1712, 1711, 1739, 1763, 1749, 1712, 1709, 1733, 1761, 1754, 1717, 1706, 1729, 1759, 1758, 1713, 1712, 1726, 1757, 1761, 1727, 1703, 1725, 1754, 1762, 1731, 1700, 1719, 1749, 1762, 1737, 1709, 1717, 1744, 1766, 1741, 1707, 1712, 1742, 1765, 1744, 1702, 1726, 1742, 1767, 1750, 1712, 1706, 1733, 1763, 1756, 1715, 1705, 1728, 1760, 1759, 1722, 1722, 1722, 1747, 1751, 1719, 1697, 1721, 1751, 1761, 1727, 1695, 1712, 1745, 17

Traceback (most recent call last):
  File "<stdin>", line 2, in <module>
KeyboardInterrupt: 


In [33]:
requests.GETRequest("http://192.168.0.37:5001/save")